In [33]:
from langchain_openai import ChatOpenAI
from crewai_tools import PDFSearchTool
from crewai.tools import tool
from crewai import Crew
from crewai import Task
from crewai import Agent
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_community.tools import TavilySearchResults
import getpass

In [2]:
from dotenv import load_dotenv
import os
load_dotenv()


True

In [3]:
llm = ChatOpenAI(
    openai_api_base="https://api.groq.com/openai/v1",
    model="llama-3.3-70b-versatile",
    api_key=os.getenv("GROQ_API_KEY"),
    temperature=1,
    max_tokens=1000
)

In [17]:
response = llm.invoke("What is the capital of Pakistan?")
print(response)

content='The capital of Pakistan is Islamabad.' response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 42, 'total_tokens': 50, 'completion_tokens_details': None, 'prompt_tokens_details': None, 'queue_time': 0.062000846, 'prompt_time': 0.002014243, 'completion_time': 0.029090909, 'total_time': 0.031105152}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_90c1d253ff', 'finish_reason': 'stop', 'logprobs': None} id='run-4f9df476-46dc-41bb-9f12-253601150fb0-0'


In [4]:
# loading pdf file
def load_pdf_file(data):
    loader = DirectoryLoader(data,
                             glob="*.pdf",
                             loader_cls=PyPDFLoader)
    
    documents = loader.load()
    return documents

In [9]:
data = load_pdf_file("data")

In [19]:
rag_tool = PDFSearchTool(pdf="data.pdf",
    config=dict(
        llm=dict(
            provider="groq", # or google, openai, anthropic, llama2, ...
            config=dict(
                model="llama-3.3-70b-versatile",
                # temperature=0.5,
                # top_p=1,
                # stream=true,
            ),
        ),
        embedder=dict(
            provider="huggingface", # or openai, ollama, ...
            config=dict(
                model="sentence-transformers/all-MiniLM-L6-v2",
                # task_type="retrieval_document",
                # title="Embeddings",
            ),
        ),
    )
)

Inserting batches in chromadb:   0%|          | 0/1 [00:00<?, ?it/s]c:\Users\Saim Safeer\Desktop\RAG\agentic_rag_ai\env\Lib\site-packages\chromadb\types.py:144: PydanticDeprecatedSince211: Accessing this attribute on the instance is deprecated, and will be removed in Pydantic V3. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x
Inserting batches in chromadb: 100%|██████████| 1/1 [00:03<00:00,  3.29s/it]


In [20]:
rag_tool.run("what is the goal reducing sequential computation?")

Using Tool: Search a PDF's content


'Relevant Content:\nRecurrent models typically factor computation along the symbol positions of the input and output sequences. Aligning the positions to steps in computation time, they generate a sequence of hidden states ht, as a function of the previous hidden state ht−1 and the input for position t. This inherently sequential nature precludes parallelization within training examples, which becomes critical at longer sequence lengths, as memory constraints limit batching across examples. Recent work has achieved signiﬁcant improvements in computational efﬁciency through factorization tricks [18] and conditional computation [26], while also improving model performance in case of the latter. The fundamental constraint of sequential computation, however, remains. Attention mechanisms have become an integral part of compelling sequence modeling and transduc- tion models in various tasks, allowing modeling of dependencies without regard to their distance in the input or output sequences 

In [37]:
os.environ["TAVILY_API_KEY"] = getpass.getpass()

In [38]:
TavilySearchResults().invoke("whats the weather in san francisco?")

[{'title': 'Friday, April 4, 2025. San Francisco, CA - Weather Forecast',
  'url': 'https://weathershogun.com/weather/usa/ca/san-francisco/480/april/2025-04-04',
  'content': 'San Francisco, California Weather: Friday, April 4, 2025. Cloudy weather, overcast skies with clouds. Day 64°. Night 50°. Precipitation 0 %.',
  'score': 0.9612766},
 {'title': 'Weather in san francisco',
  'url': 'https://www.weatherapi.com/',
  'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1743745312, 'localtime': '2025-04-03 22:41'}, 'current': {'last_updated_epoch': 1743744600, 'last_updated': '2025-04-03 22:30', 'temp_c': 11.7, 'temp_f': 53.1, 'is_day': 0, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png', 'code': 1003}, 'wind_mph': 4.9, 'wind_kph': 7.9, 'wind_degree': 291, 'wind_dir': 'WNW', 'pressure_mb': 1019

In [ ]:
@tool
def router_tool(question):
    if 'self-attention' in question:
        return rag_tool.run(question)
    else:
        return TavilySearchResults().invoke(question)